In [2]:
from pyspark.sql import SparkSession

# Start Spark session
spark = SparkSession.builder.appName("NaiveBayes_Example").getOrCreate()

# Create a 100% predictable dataset
df = spark.createDataFrame([
    ("I feel great", "pos", "cancer"),
    ("This is amazing", "pos", "non-cancer"),
    ("Not good at all", "neg", "cancer"),
    ("Terrible experience", "neg", "non-cancer")
], ["text", "sentiment", "cancer"])

df.show()


from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline

# Tokenizer and stop words removal
tokenizer = Tokenizer(inputCol="text", outputCol="tokens")
stopwords_remover = StopWordsRemover(inputCol="tokens", outputCol="filtered_tokens")

# Vectorization and TF-IDF
count_vectorizer = CountVectorizer(inputCol="filtered_tokens", outputCol="raw_features")
idf = IDF(inputCol="raw_features", outputCol="features")

# Label encoding
indexer = StringIndexer(inputCol="sentiment", outputCol="label")

# Build pipeline
pipeline = Pipeline(stages=[tokenizer, stopwords_remover, count_vectorizer, idf, indexer])

# Fit and transform
processed_df = pipeline.fit(df).transform(df)

# Split the data (use all data for training for simplicity)
train_data, test_data = processed_df.randomSplit([0.8, 0.2], seed=42)

# Train Naive Bayes model
nb = NaiveBayes(featuresCol="features", labelCol="label", modelType="multinomial")
nb_model = nb.fit(train_data)

# Predict
predictions = nb_model.transform(test_data)

# Evaluate
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print(f"Test Accuracy: {accuracy:.2f}")


StatementMeta(ba5e360d-f184-47a0-9859-76b5031b79e3, 36, 7, Finished, Available, Finished)

+-------------------+---------+----------+
|               text|sentiment|    cancer|
+-------------------+---------+----------+
|       I feel great|      pos|    cancer|
|    This is amazing|      pos|non-cancer|
|    Not good at all|      neg|    cancer|
|Terrible experience|      neg|non-cancer|
+-------------------+---------+----------+

Test Accuracy: 1.00
